In [1]:
import os

os.environ['HF_HOME']

'/projects/academic/rohini/m44/software/2023.01/hf-home'

In [2]:
from utils import TextGenerator

text_gen = TextGenerator('mistralai/Mistral-7B-Instruct-v0.1', load_in_4bit=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Generate Dispositions

In [ ]:
disp_task_que = ''
disp_system_message = "You are an expet psychologist who excels in describing a person's disposition"

In [ ]:
import pathlib

disp_inst = pathlib.Path('/projects/academic/rohini/m44/misinfo-proj/datasets/inst-templates/expt2/disposition-gen.txt').read_text()

In [ ]:
dispositions = text_gen(disp_system_message, disp_task_que, disp_inst)
print(dispositions)

## Generate Conversations

In [ ]:
task_que = '(conversation)'
system_message = "You are an expert dialog writer. You write compelling conversations with characters that have rich, well developed personalities."

In [ ]:
import pandas as pd


df = pd.read_csv("/projects/academic/rohini/m44/misinfo-proj/datasets/scraped_crowdsource_data.tsv", delimiter='\t')

In [ ]:
tweet = df.iloc[0]['tweet']

In [ ]:
import random
import pathlib

# Choosing the misinformed person's personality
disposition_dir = pathlib.Path('/projects/academic/rohini/m44/misinfo-proj/datasets/inst-templates/expt2/disposition')
disposition = random.choice(sorted(disposition_dir.glob('*.txt')))
disposition = disposition.read_text()

In [ ]:
inst_path = pathlib.Path('/projects/academic/rohini/m44/misinfo-proj/datasets/inst-templates/expt2/conv-gen.txt')
inst = inst_path.read_text()

# Add the tweet to the `inst`
inst = inst.format(disposition, tweet)
print(inst)

In [ ]:
op_step1 = text_gen(system_message, task_que, inst)
print(op_step1)

### Generate the next few turns

In [ ]:
new_inst = f"Given the dialog (history), generate the next 4 turns of a possibly longer (conversation).\n\n(history)\n{op_step1}"

In [ ]:
op_step2 = text_gen(system_message, task_que, new_inst)
print(op_step2)

## Batch Generate

In [ ]:
from utils import BatchTextGenerator

text_gen = BatchTextGenerator('meta-llama/Llama-2-13b-chat-hf', load_in_4bit=False)

In [ ]:
batch_size = 16
tweets = df.sample(n=batch_size)[['tweet']]

In [ ]:
import random
import pathlib

# Choosing the misinformed person's personality
disposition_dir = pathlib.Path('/projects/academic/rohini/m44/misinfo-proj/datasets/inst-templates/expt2/disposition')
dispositions = random.choices(sorted(disposition_dir.glob('*.txt')), k=batch_size)
dispositions = [disp.read_text() for disp in dispositions]
# print(dispositions)

In [ ]:
inst_path = pathlib.Path('/projects/academic/rohini/m44/misinfo-proj/datasets/inst-templates/expt2/conv-gen.txt')
inst = inst_path.read_text()

instructions = [
    inst.format(disp, tweet)
        for disp, (id, tweet) in zip(dispositions, tweets.itertuples())
]


In [ ]:
batch_op = text_gen(system_message, task_que, instructions)
print(batch_op)